# <b>Train the Network</b>

In [1]:
#!/usr/bin/python3
SEED = 1234

import numpy as np
np.random.seed(SEED)

from numpy.random import randint, choice

from keras.models import Sequential
#from keras.optimizers import SGD
from keras.layers import Dense

from sklearn.model_selection import train_test_split


Nmin, Nmax = 0, 100
L = 5
Ntot = 100000

def preprocess_forward(l: list) -> list:
    # recieves a list of integers from Nmin to Nmax
    # and returns a list of floats from 0 to 1
    return [float(_)/Nmax for _ in l]


def preprocess_backward(l: list) -> list:
    # recieves a list of floats from 0 to 1
    # and returns a list of integers from Nmin to Nmax
    return [int(round(_*Nmax)) for _ in l]

def compute_correct_value(l: list) -> list:
    # recieves a list of integers from Nmin to Nmax
    # outputs a list of binaries (0,1) of size L**2
    base = [0 for _ in range(L**2)]
    l_sorted = np.asarray(sorted(l))
    for i,b_i in enumerate(l):
        i_local = (l_sorted == b_i).nonzero()[0][0]
        base[i * L + i_local] = 1
    return base    

class NNwork():
    def __init__(self, depth: int=3, neurons: int=4, 
                        activation: str="relu") -> Sequential:
        self.network = Sequential([
                        Dense(neurons, 
                              activation=activation,
                              ) for _ in range(depth)] +
                        [Dense(L**2, activation="sigmoid")])
        self.training_history = None

    def train(self, train_data, val_data,
                    epochs=20, batch_size=32, optimizer="sgd") -> None:
        self.network.compile(optimizer=optimizer,
                            loss="binary_crossentropy",
                            metrics=["accuracy"])
        self.training_history = self.network.fit(*train_data, 
                                        validation_data=val_data,
                                        epochs=epochs,
                                        batch_size=batch_size).history

    def predict(self, x_data):
        return self.network.predict(x_data)

# Generate the database: "Ntot" lists of unrepeated "L" numbers each
numbers = np.linspace(Nmin, Nmax, Nmax-Nmin+1).astype('int')
database = [choice(numbers, L, replace=False).tolist() for _ in range(Ntot)]

# Preprocess the database by scaling it
preprocessed_database = [preprocess_forward(item) for item in database]

# Check the consistency of the pre-post processing operations
assert(database==[preprocess_backward(item) for item in preprocessed_database])
print("\ninfo: Post(Pre(data)) == data evaluated to True\n")

# Generate the correct answers by sorting it
correct_answers = [compute_correct_value(item) for item in preprocessed_database]

# Split into training, validation, and testing
X_train, X_test, Y_train, Y_test = train_test_split(preprocessed_database, 
                                                    correct_answers,
                                                    test_size=0.3,
                                                    random_state=SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test,
                                                test_size=0.5, 
                                                random_state=SEED)


# Instantiate a neural network and train it
network = NNwork(depth=4, neurons=32, activation="relu")
network.train((X_train, Y_train), (X_val, Y_val),
                epochs=100, batch_size=16, optimizer="SGD")

# Define a binarizer for predictions
def binarizer(ypred):
    result = []
    for y in ypred:
        local_result = [0 for _ in range(L**2)]
        for i in range(L):
            ix = np.argmax(y[i*L:(i+1)*L])
            local_result[i*L+ix] = 1
        result += [local_result]
    return result


# Get predictions over all the datasets to compare with another metric
Y_train_pred = binarizer(network.predict(X_train))
Y_val_pred = binarizer(network.predict(X_val))
Y_test_pred = binarizer(network.predict(X_test))


# Define a metric report
def report_metric(ytrue, ypred, label):
    #convert float [0,1] predictions into binary {0,1}
    correct, incorrect = 0,0
    print(ytrue[0], ypred[0])
    for i,y in enumerate(ytrue):
        if i<10:
            print(y, ypred[i])
        if y==ypred[i]:
            correct +=1
        else:
            incorrect += 1
    print(f"For the '{label}' dataset, accuracy is "\
            f"{round(100*correct/(correct+incorrect), 2)}%.\n"\
            f"correct: {correct}\nINcorrect: {incorrect}")


# Report metrics:
report_metric(Y_train, Y_train_pred, label="train")
report_metric(Y_val, Y_val_pred, label="val")
report_metric(Y_test, Y_test_pred, label="test")



info: Post(Pre(data)) == data evaluated to True

Epoch 1/100
4375/4375 [==============================] - 19s 4ms/step - loss: 0.5348 - accuracy: 0.1380 - val_loss: 0.4959 - val_accuracy: 0.0583
Epoch 2/100
4375/4375 [==============================] - 13s 3ms/step - loss: 0.4891 - accuracy: 0.0795 - val_loss: 0.4796 - val_accuracy: 0.0343
Epoch 3/100
4375/4375 [==============================] - 14s 3ms/step - loss: 0.4653 - accuracy: 0.0524 - val_loss: 0.4504 - val_accuracy: 0.0758
Epoch 4/100
4375/4375 [==============================] - 13s 3ms/step - loss: 0.4406 - accuracy: 0.1033 - val_loss: 0.4319 - val_accuracy: 0.1263
Epoch 5/100
4375/4375 [==============================] - 14s 3ms/step - loss: 0.4224 - accuracy: 0.1258 - val_loss: 0.4093 - val_accuracy: 0.1191
Epoch 6/100
4375/4375 [==============================] - 13s 3ms/step - loss: 0.3982 - accuracy: 0.1138 - val_loss: 0.3898 - val_accuracy: 0.1146
Epoch 7/100
4375/4375 [==============================] - 13s 3ms/step - lo

# <b>Try the network</b>

In [14]:
# Define an utility function
def transform_prediction_ordering(x, nnetwork):
  ypred = binarizer(nnetwork.predict([preprocess_forward(x)]))[0]
  result = [-1 for _ in range(L)]
  for i in range(L):
    result[np.argmax(ypred[i*L:(i+1)*L])] = x[i]
  return result

In [17]:
# Define lists to be tested
TEST_LISTS = [[16,20,6,4,12],
              [23,0,44,79,6],
              [18,64,5,92,88],
              [2,19,12,42,5]]

for TEST_LIST in TEST_LISTS:
  NETWORK_RESULT = transform_prediction_ordering(TEST_LIST, network)
  # Print the results
  print(f'We gave the network:\n{TEST_LIST}\nand it returned the following list:\n{NETWORK_RESULT}\n\n')

We gave the network:
[16, 20, 6, 4, 12]
and it returned the following list:
[4, 6, 12, 16, 20]


We gave the network:
[23, 0, 44, 79, 6]
and it returned the following list:
[0, 6, 23, 44, 79]


We gave the network:
[18, 64, 5, 92, 88]
and it returned the following list:
[5, 18, 64, 88, 92]


We gave the network:
[2, 19, 12, 42, 5]
and it returned the following list:
[2, 5, 12, 19, 42]


